## 영화진흥위원회 OPENAPI로 감독데이터 불러오기

In [6]:
# openapi 함수
import requests
import json

def get_movieInfo(movieCd):
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=001a81c336fe1093a7b65899f0c4e9d4&movieCd='+str(movieCd)
    res = requests.get(url)
    text = res.text
    #print(text)
    
    d = json.loads(text)

    movieInfo = d['movieInfoResult']['movieInfo']

    return movieInfo

In [7]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [8]:
total = pd.read_csv('./datasets/배우감독파생변수용.csv')

In [9]:
## 대표코드 리스트
list = total['대표코드'].tolist()

In [10]:
# 전체 데이터 917개의 영화 별 감독데이터 불러오는 함수
b = []
for i in list:
    movieInfo = get_movieInfo(i)
    a = movieInfo["directors"]
    a_1 = pd.DataFrame(a)
    b.append(a_1)

In [ ]:
# kk = []
# for i in range(860, len(b)):
#     print(i)
#     bb = b[i]['peopleNm']
#     kk.append(bb)

In [16]:
# 감독데이터가 없는 컬럼 확인 후 빈 컬럼값 생성
b[859] = pd.DataFrame(columns = ['peopleNm','peopleNmEn'])

In [ ]:
# 감독데이터중 배우한글데이터 + 상위 2개 감독만 추출하겠다.
kkk =[]
for i in range(0, len(b)):
    array_1 = np.array(b[i]['peopleNm'].iloc[:1])
    kkk.append(array_1)

In [35]:
# 데이터프레임에 저장
directors = pd.DataFrame(kkk)

In [21]:
directors.to_csv('./datasets/감독.csv')

In [36]:
df = pd.read_csv('./datasets/감독.csv', index_col=0)
df

,0,1
0,김한민,NaN
1,이병헌,NaN
2,김용화,NaN
3,윤제균,NaN
4,안소니 루소,조 루소
...,...,...
912,스티븐 스필버그,NaN
913,권형진,NaN
914,스티븐 쿼일,NaN
915,안젤리나 졸리,NaN


In [37]:
# 0~2 컬럼 value_counts 값
a = df['0'].value_counts()

In [38]:
# 데이터프레임화
a = pd.DataFrame(a)

In [39]:
# 영화별 배우 출현 횟수 총 합
aaa = a.T.sum().sort_values(ascending=False)

In [40]:
# 데이터프레임화
aaa = pd.DataFrame(aaa)
aaa

,0
이준익,7
시즈노 코분,6
유야마 쿠니히코,5
제임스 완,5
폴 W.S. 앤더슨,5
...,...
구수환,1
도미닉 세나,1
웨인 쏜리,1
롭 민코프,1


In [42]:
for i in range(583):
    for j in range(917):
        if df['0'].values[j] == aaa.index[i]:
           df['0'].values[j] = int(aaa.values[i])

In [46]:
df = df[['0']]

In [47]:
감독가중치 = pd.DataFrame(df.T.sum())
감독가중치.columns = ['감독가중치']
감독가중치

,감독가중치
0,2
1,3
2,3
3,1
4,3
...,...
912,3
913,1
914,2
915,1


In [48]:
ls = []

for i in range(917):
    if total['전국 관객수'][i] >= 10000000:
        ls.append(1)
    elif (total['전국 관객수'][i] < 10000000) and (total['전국 관객수'][i] >= 5000000):
        ls.append(10)
    elif (total['전국 관객수'][i] < 5000000) and (total['전국 관객수'][i] >= 1000000):
        ls.append(20)
    else:
        ls.append(30)

In [49]:
가중치값 = pd.DataFrame(ls)
가중치값.columns = ['가중치값']

In [51]:
감독data = pd.concat([감독가중치, 가중치값], axis=1)

In [52]:
감독가중치['감독가중치'] = 감독data['감독가중치'] / 감독data['가중치값']

In [53]:
감독가중치

,감독가중치
0,2.0
1,3.0
2,3.0
3,1.0
4,3.0
...,...
912,0.1
913,0.033333
914,0.066667
915,0.033333


In [54]:
감독가중치.to_csv('./datasets/감독가중치.csv')